# Protein Phylogenetic Analysis Pipeline
## Efficient homolog search, alignment, and tree construction using PDB structures

### Features:
- GPU-accelerated MMseqs2 homolog search
- FAMSA2 ultra-fast alignment
- IQ-TREE phylogenetic inference
- Comprehensive logging and error handling
- Google Drive integration for results
- Interactive visualizations and CSV outputs

## CELL 1: Complete Installation Suite with Logging
All installations happen here with explicit error catching and verbose logging

In [ ]:
# ============================================================================
# COMPREHENSIVE INSTALLATION CELL WITH LOGGING
# ============================================================================

import subprocess
import sys
import os
from datetime import datetime
import json

# Initialize installation log
install_log = []
install_start = datetime.now()

def log_install(package, status, message="", error=None):
    """Log installation attempts with timestamp"""
    entry = {
        'timestamp': datetime.now().isoformat(),
        'package': package,
        'status': status,
        'message': message,
        'error': str(error) if error else None
    }
    install_log.append(entry)
    
    # Print colored output
    if status == 'success':
        print(f"✅ {package}: {message}")
    elif status == 'failed':
        print(f"❌ {package}: {message}")
        if error:
            print(f"   Error details: {error}")
    else:
        print(f"🔄 {package}: {message}")
    return entry

def safe_install(package_name, pip_package=None, test_import=None):
    """Safely install a package with error handling"""
    pip_package = pip_package or package_name
    test_import = test_import or package_name.replace('-', '_')
    
    try:
        log_install(package_name, 'installing', f'Starting installation of {pip_package}')
        
        # Run pip install
        result = subprocess.run(
            [sys.executable, '-m', 'pip', 'install', pip_package, '--upgrade'],
            capture_output=True,
            text=True,
            timeout=300
        )
        
        if result.returncode != 0:
            raise Exception(f"pip install failed: {result.stderr}")
        
        # Test import
        exec(f"import {test_import}")
        
        # Get version if possible
        try:
            version = subprocess.run(
                [sys.executable, '-c', f"import {test_import}; print({test_import}.__version__)"],
                capture_output=True,
                text=True
            ).stdout.strip()
            log_install(package_name, 'success', f'Installed version {version}')
        except:
            log_install(package_name, 'success', 'Installed (version unknown)')
        
        return True
        
    except subprocess.TimeoutExpired:
        log_install(package_name, 'failed', 'Installation timeout', 'Timeout after 300 seconds')
        return False
    except Exception as e:
        log_install(package_name, 'failed', 'Installation failed', e)
        return False

print("="*70)
print("STARTING COMPREHENSIVE INSTALLATION PROCESS")
print(f"Start time: {install_start}")
print(f"Python version: {sys.version}")
print("="*70)
print()

# ============================================================================
# CORE SCIENTIFIC PACKAGES
# ============================================================================
print("\n📦 Installing Core Scientific Packages...")
print("-"*50)

core_packages = [
    ('numpy', 'numpy', 'numpy'),
    ('pandas', 'pandas', 'pandas'),
    ('matplotlib', 'matplotlib', 'matplotlib'),
    ('seaborn', 'seaborn', 'seaborn'),
    ('scipy', 'scipy', 'scipy'),
]

for package, pip_name, import_name in core_packages:
    safe_install(package, pip_name, import_name)

# ============================================================================
# BIOINFORMATICS PACKAGES
# ============================================================================
print("\n🧬 Installing Bioinformatics Packages...")
print("-"*50)

bio_packages = [
    ('biopython', 'biopython', 'Bio'),
    ('pyfamsa', 'pyfamsa', 'pyfamsa'),
    ('dendropy', 'dendropy', 'dendropy'),
    ('ete3', 'ete3', 'ete3'),
]

for package, pip_name, import_name in bio_packages:
    safe_install(package, pip_name, import_name)

# ============================================================================
# PHYLOGENETIC ANALYSIS PACKAGES
# ============================================================================
print("\n🌳 Installing Phylogenetic Analysis Packages...")
print("-"*50)

# Special handling for piqtree (IQ-TREE Python wrapper)
try:
    log_install('piqtree', 'installing', 'Attempting to install IQ-TREE Python wrapper')
    result = subprocess.run(
        [sys.executable, '-m', 'pip', 'install', 'piqtree'],
        capture_output=True,
        text=True,
        timeout=300
    )
    
    if result.returncode == 0:
        log_install('piqtree', 'success', 'IQ-TREE Python wrapper installed')
    else:
        # Fallback: install IQ-TREE binary directly
        log_install('piqtree', 'warning', 'piqtree not available, will use IQ-TREE binary')
except Exception as e:
    log_install('piqtree', 'warning', f'Will use IQ-TREE binary instead: {e}')

# ============================================================================
# VISUALIZATION AND UI PACKAGES
# ============================================================================
print("\n🎨 Installing Visualization and UI Packages...")
print("-"*50)

viz_packages = [
    ('plotly', 'plotly', 'plotly'),
    ('ipywidgets', 'ipywidgets', 'ipywidgets'),
    ('tqdm', 'tqdm', 'tqdm'),
]

for package, pip_name, import_name in viz_packages:
    safe_install(package, pip_name, import_name)

# ============================================================================
# UTILITY PACKAGES
# ============================================================================
print("\n🔧 Installing Utility Packages...")
print("-"*50)

utility_packages = [
    ('requests', 'requests', 'requests'),
    ('wget', 'wget', 'wget'),
    ('gdown', 'gdown', 'gdown'),
]

for package, pip_name, import_name in utility_packages:
    safe_install(package, pip_name, import_name)

# ============================================================================
# BINARY TOOLS INSTALLATION
# ============================================================================
print("\n⚙️ Installing Binary Tools...")
print("-"*50)

# Install MMseqs2
try:
    log_install('MMseqs2', 'installing', 'Downloading MMseqs2 binary')
    
    # Create tools directory
    os.makedirs('/content/tools', exist_ok=True)
    
    # Download MMseqs2
    mmseqs_url = 'https://github.com/soedinglab/MMseqs2/releases/latest/download/mmseqs-linux-avx2.tar.gz'
    
    result = subprocess.run(
        f'cd /content/tools && wget -q {mmseqs_url} && tar xzf mmseqs-linux-avx2.tar.gz',
        shell=True,
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0:
        # Test MMseqs2
        test = subprocess.run(
            '/content/tools/mmseqs/bin/mmseqs version',
            shell=True,
            capture_output=True,
            text=True
        )
        version = test.stdout.strip() if test.returncode == 0 else 'unknown'
        log_install('MMseqs2', 'success', f'Binary installed, version: {version}')
    else:
        raise Exception(f"Download failed: {result.stderr}")
        
except Exception as e:
    log_install('MMseqs2', 'failed', 'Could not install MMseqs2 binary', e)

# Install IQ-TREE
try:
    log_install('IQ-TREE', 'installing', 'Downloading IQ-TREE binary')
    
    iqtree_url = 'https://github.com/iqtree/iqtree2/releases/download/v2.3.6/iqtree-2.3.6-Linux-intel.tar.gz'
    
    result = subprocess.run(
        f'cd /content/tools && wget -q {iqtree_url} && tar xzf iqtree-2.3.6-Linux-intel.tar.gz',
        shell=True,
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0:
        # Create symlink for easier access
        subprocess.run('ln -sf /content/tools/iqtree-2.3.6-Linux-intel/bin/iqtree2 /content/tools/iqtree2', shell=True)
        
        # Test IQ-TREE
        test = subprocess.run(
            '/content/tools/iqtree2 --version',
            shell=True,
            capture_output=True,
            text=True
        )
        version = test.stdout.split('\n')[0] if test.returncode == 0 else 'unknown'
        log_install('IQ-TREE', 'success', f'Binary installed: {version}')
    else:
        raise Exception(f"Download failed: {result.stderr}")
        
except Exception as e:
    log_install('IQ-TREE', 'failed', 'Could not install IQ-TREE binary', e)

# ============================================================================
# VERIFY GPU AVAILABILITY
# ============================================================================
print("\n🎮 Checking GPU Availability...")
print("-"*50)

try:
    gpu_check = subprocess.run(
        'nvidia-smi --query-gpu=name,memory.total --format=csv,noheader',
        shell=True,
        capture_output=True,
        text=True
    )
    
    if gpu_check.returncode == 0:
        gpu_info = gpu_check.stdout.strip()
        log_install('GPU', 'success', f'GPU detected: {gpu_info}')
    else:
        log_install('GPU', 'warning', 'No GPU detected, will use CPU')
except Exception as e:
    log_install('GPU', 'warning', f'Could not check GPU: {e}')

# ============================================================================
# MOUNT GOOGLE DRIVE
# ============================================================================
print("\n☁️ Mounting Google Drive...")
print("-"*50)

try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Create log directory
    log_dir = '/content/drive/MyDrive/phylo_logs'
    os.makedirs(log_dir, exist_ok=True)
    
    # Save installation log
    log_file = f'{log_dir}/installation_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
    with open(log_file, 'w') as f:
        json.dump(install_log, f, indent=2)
    
    log_install('Google Drive', 'success', f'Mounted and log saved to {log_file}')
    
except Exception as e:
    log_install('Google Drive', 'failed', 'Could not mount Google Drive', e)
    print("⚠️ Continuing without Drive access, results will be saved locally")

# ============================================================================
# INSTALLATION SUMMARY
# ============================================================================
print("\n" + "="*70)
print("INSTALLATION SUMMARY")
print("="*70)

# Count successes and failures
success_count = sum(1 for entry in install_log if entry['status'] == 'success')
failed_count = sum(1 for entry in install_log if entry['status'] == 'failed')
warning_count = sum(1 for entry in install_log if entry['status'] == 'warning')

print(f"✅ Successful installations: {success_count}")
print(f"⚠️ Warnings: {warning_count}")
print(f"❌ Failed installations: {failed_count}")
print(f"⏱️ Total time: {datetime.now() - install_start}")

if failed_count > 0:
    print("\n❌ Failed packages:")
    for entry in install_log:
        if entry['status'] == 'failed':
            print(f"  - {entry['package']}: {entry['message']}")

print("\n✅ Installation phase complete. Proceed to imports.")

## CELL 2: Import All Libraries with Validation

In [ ]:
# ============================================================================
# COMPREHENSIVE IMPORTS WITH VALIDATION
# ============================================================================

import_log = []
import_start = datetime.now()

def safe_import(module_name, import_as=None, from_module=None):
    """Safely import a module with error handling"""
    try:
        if from_module:
            exec(f"from {from_module} import {module_name}")
            globals()[module_name] = eval(module_name)
        elif import_as:
            exec(f"import {module_name} as {import_as}")
            globals()[import_as] = eval(import_as)
        else:
            exec(f"import {module_name}")
            globals()[module_name] = eval(module_name)
        
        # Get version if available
        version_str = "unknown"
        try:
            if import_as:
                version_str = eval(f"{import_as}.__version__")
            else:
                version_str = eval(f"{module_name}.__version__")
        except:
            pass
        
        import_log.append({'module': module_name, 'status': 'success', 'version': version_str})
        print(f"✅ Imported {module_name} (version: {version_str})")
        return True
        
    except Exception as e:
        import_log.append({'module': module_name, 'status': 'failed', 'error': str(e)})
        print(f"❌ Failed to import {module_name}: {e}")
        return False

print("="*70)
print("IMPORTING ALL REQUIRED LIBRARIES")
print("="*70)
print()

# ============================================================================
# STANDARD LIBRARY IMPORTS
# ============================================================================
print("📚 Standard Library Imports...")
print("-"*50)

safe_import('os')
safe_import('sys')
safe_import('json')
safe_import('logging')
safe_import('subprocess')
safe_import('shutil')
safe_import('tempfile')
safe_import('warnings')
safe_import('traceback')
safe_import('datetime', from_module='datetime')
safe_import('Path', from_module='pathlib')
safe_import('defaultdict', from_module='collections')
safe_import('Counter', from_module='collections')

# ============================================================================
# SCIENTIFIC COMPUTING IMPORTS
# ============================================================================
print("\n🔬 Scientific Computing Libraries...")
print("-"*50)

safe_import('numpy', import_as='np')
safe_import('pandas', import_as='pd')
safe_import('scipy')
safe_import('stats', from_module='scipy')

# ============================================================================
# VISUALIZATION IMPORTS
# ============================================================================
print("\n📊 Visualization Libraries...")
print("-"*50)

safe_import('matplotlib.pyplot', import_as='plt')
safe_import('seaborn', import_as='sns')
safe_import('plotly.graph_objects', import_as='go')
safe_import('plotly.express', import_as='px')

# Set visualization defaults
try:
    sns.set_style('whitegrid')
    plt.rcParams['figure.figsize'] = (12, 8)
    plt.rcParams['font.size'] = 10
    warnings.filterwarnings('ignore')
    print("✅ Visualization defaults configured")
except Exception as e:
    print(f"⚠️ Could not set visualization defaults: {e}")

# ============================================================================
# BIOINFORMATICS IMPORTS
# ============================================================================
print("\n🧬 Bioinformatics Libraries...")
print("-"*50)

# Biopython imports
safe_import('Bio')
safe_import('SeqIO', from_module='Bio')
safe_import('Seq', from_module='Bio.Seq')
safe_import('SeqRecord', from_module='Bio.SeqRecord')
safe_import('PDBList', from_module='Bio.PDB')
safe_import('PDBParser', from_module='Bio.PDB')
safe_import('AlignIO', from_module='Bio')
safe_import('Phylo', from_module='Bio')
safe_import('DistanceCalculator', from_module='Bio.Phylo.TreeConstruction')
safe_import('DistanceTreeConstructor', from_module='Bio.Phylo.TreeConstruction')

# FAMSA alignment
try:
    safe_import('pyfamsa')
except:
    print("⚠️ pyfamsa not available, will use fallback alignment method")

# Phylogenetic analysis
safe_import('dendropy')
try:
    safe_import('ete3')
except:
    print("⚠️ ete3 not available, will use alternative tree visualization")

# ============================================================================
# UI AND UTILITY IMPORTS
# ============================================================================
print("\n🛠️ UI and Utility Libraries...")
print("-"*50)

safe_import('widgets', from_module='ipywidgets')
safe_import('display', from_module='IPython.display')
safe_import('HTML', from_module='IPython.display')
safe_import('tqdm.notebook', import_as='tqdm_notebook')
safe_import('requests')
safe_import('wget')

# ============================================================================
# GOOGLE COLAB SPECIFIC IMPORTS
# ============================================================================
print("\n☁️ Google Colab Utilities...")
print("-"*50)

try:
    safe_import('drive', from_module='google.colab')
    safe_import('files', from_module='google.colab')
    IN_COLAB = True
    print("✅ Running in Google Colab environment")
except:
    IN_COLAB = False
    print("⚠️ Not in Colab environment, some features may be limited")

# ============================================================================
# SETUP LOGGING SYSTEM
# ============================================================================
print("\n📝 Setting up Logging System...")
print("-"*50)

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s | %(levelname)s | %(funcName)s | Line:%(lineno)d | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Create main logger
logger = logging.getLogger('phylo_pipeline')
logger.setLevel(logging.DEBUG)

# Remove default handler to avoid duplication
logger.handlers = []

# Console handler
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(logging.INFO)
console_formatter = logging.Formatter(
    '%(asctime)s | %(levelname)s | %(message)s',
    datefmt='%H:%M:%S'
)
console_handler.setFormatter(console_formatter)
logger.addHandler(console_handler)

# File handler (if Drive is mounted)
if IN_COLAB and os.path.exists('/content/drive/MyDrive'):
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_file = f'/content/drive/MyDrive/phylo_logs/run_{timestamp}.log'
    os.makedirs(os.path.dirname(log_file), exist_ok=True)
    
    file_handler = logging.FileHandler(log_file)
    file_handler.setLevel(logging.DEBUG)
    file_formatter = logging.Formatter(
        '%(asctime)s | %(levelname)s | %(funcName)s | Line:%(lineno)d | %(message)s'
    )
    file_handler.setFormatter(file_formatter)
    logger.addHandler(file_handler)
    
    print(f"✅ Logging to file: {log_file}")
else:
    print("⚠️ File logging disabled (Drive not available)")

logger.info("Logging system initialized")

# ============================================================================
# IMPORT SUMMARY
# ============================================================================
print("\n" + "="*70)
print("IMPORT SUMMARY")
print("="*70)

success_imports = sum(1 for entry in import_log if entry['status'] == 'success')
failed_imports = sum(1 for entry in import_log if entry['status'] == 'failed')

print(f"✅ Successful imports: {success_imports}")
print(f"❌ Failed imports: {failed_imports}")
print(f"⏱️ Import time: {datetime.now() - import_start}")

if failed_imports > 0:
    print("\n❌ Failed imports (may affect functionality):")
    for entry in import_log:
        if entry['status'] == 'failed':
            print(f"  - {entry['module']}: {entry.get('error', 'Unknown error')}")
    print("\n⚠️ Some features may be limited due to import failures")
else:
    print("\n✅ All imports successful! Ready to proceed.")

# Save import log if Drive is available
if IN_COLAB and os.path.exists('/content/drive/MyDrive'):
    import_log_file = f'/content/drive/MyDrive/phylo_logs/imports_{timestamp}.json'
    with open(import_log_file, 'w') as f:
        json.dump(import_log, f, indent=2)
    print(f"\n💾 Import log saved to: {import_log_file}")

## CELL 3: Helper Functions and Exception Handling Framework

In [ ]:
# ============================================================================
# HELPER FUNCTIONS AND EXCEPTION HANDLING FRAMEWORK
# ============================================================================

logger.info("Setting up helper functions and exception handling framework")

# ============================================================================
# EXCEPTION HANDLING DECORATORS
# ============================================================================

def safe_cell_execution(func_name):
    """Decorator for cell-level exception handling"""
    def decorator(func):
        def wrapper(*args, **kwargs):
            try:
                logger.info(f"Starting cell operation: {func_name}")
                result = func(*args, **kwargs)
                logger.info(f"Successfully completed: {func_name}")
                return result
            except Exception as e:
                logger.error(f"Error in {func_name}: {str(e)}", exc_info=True)
                print(f"\n❌ Error in {func_name}: {str(e)}")
                print("Check log file for full traceback")
                
                # Save error details to Drive if available
                if IN_COLAB and os.path.exists('/content/drive/MyDrive'):
                    error_file = f'/content/drive/MyDrive/phylo_logs/error_{timestamp}_{func_name}.txt'
                    with open(error_file, 'w') as f:
                        f.write(f"Error in {func_name}\n")
                        f.write(f"Time: {datetime.now()}\n")
                        f.write(f"Error: {str(e)}\n\n")
                        f.write("Full traceback:\n")
                        f.write(traceback.format_exc())
                    print(f"📝 Error details saved to: {error_file}")
                
                raise
        return wrapper
    return decorator

def safe_operation(op_name):
    """Decorator for operation-level exception handling"""
    def decorator(func):
        def wrapper(*args, **kwargs):
            try:
                logger.debug(f"Executing operation: {op_name}")
                result = func(*args, **kwargs)
                logger.debug(f"Operation successful: {op_name}")
                return result
            except Exception as e:
                logger.warning(f"Operation failed {op_name}: {e}")
                return None
        return wrapper
    return decorator

# ============================================================================
# FILE MANAGEMENT UTILITIES
# ============================================================================

class FileManager:
    """Centralized file management with logging"""
    
    def __init__(self, pdb_id, timestamp):
        self.pdb_id = pdb_id
        self.timestamp = timestamp
        self.setup_directories()
    
    def setup_directories(self):
        """Create directory structure"""
        if IN_COLAB and os.path.exists('/content/drive/MyDrive'):
            self.base_dir = f'/content/drive/MyDrive/phylo_results/{self.timestamp}_{self.pdb_id}'
        else:
            self.base_dir = f'/content/phylo_results/{self.timestamp}_{self.pdb_id}'
        
        self.dirs = {
            'logs': f'{self.base_dir}/logs',
            'sequences': f'{self.base_dir}/sequences',
            'alignments': f'{self.base_dir}/alignments',
            'trees': f'{self.base_dir}/trees',
            'visualizations': f'{self.base_dir}/visualizations',
            'csv_outputs': f'{self.base_dir}/csv_outputs',
            'checkpoints': f'{self.base_dir}/checkpoints'
        }
        
        for dir_name, dir_path in self.dirs.items():
            os.makedirs(dir_path, exist_ok=True)
            logger.debug(f"Created directory: {dir_path}")
    
    def save_file(self, content, filename, subdir=''):
        """Save file with logging"""
        if subdir and subdir in self.dirs:
            filepath = f"{self.dirs[subdir]}/{filename}"
        else:
            filepath = f"{self.base_dir}/{filename}"
        
        try:
            if isinstance(content, str):
                with open(filepath, 'w') as f:
                    f.write(content)
            elif isinstance(content, bytes):
                with open(filepath, 'wb') as f:
                    f.write(content)
            elif isinstance(content, pd.DataFrame):
                content.to_csv(filepath, index=False)
            else:
                with open(filepath, 'w') as f:
                    json.dump(content, f, indent=2)
            
            logger.info(f"Saved file: {filepath}")
            return filepath
        except Exception as e:
            logger.error(f"Failed to save {filename}: {e}")
            return None

# ============================================================================
# PROGRESS TRACKING
# ============================================================================

class ProgressTracker:
    """Track and display pipeline progress"""
    
    def __init__(self):
        self.steps = [
            'PDB Download',
            'Sequence Extraction',
            'Homolog Search',
            'Alignment',
            'Tree Construction',
            'Visualization',
            'Report Generation'
        ]
        self.current_step = 0
        self.step_times = {}
    
    def start_step(self, step_name):
        """Mark step as started"""
        self.step_times[step_name] = {'start': datetime.now()}
        logger.info(f"Starting step: {step_name}")
        print(f"\n{'='*70}")
        print(f"🔄 STEP {self.current_step + 1}/{len(self.steps)}: {step_name}")
        print(f"{'='*70}")
    
    def complete_step(self, step_name):
        """Mark step as completed"""
        if step_name in self.step_times:
            self.step_times[step_name]['end'] = datetime.now()
            duration = self.step_times[step_name]['end'] - self.step_times[step_name]['start']
            logger.info(f"Completed step: {step_name} (Duration: {duration})")
            print(f"✅ Completed: {step_name} (Time: {duration})")
            self.current_step += 1
    
    def get_summary(self):
        """Get execution summary"""
        summary = []
        for step_name, times in self.step_times.items():
            if 'end' in times:
                duration = times['end'] - times['start']
                summary.append({
                    'Step': step_name,
                    'Duration': str(duration),
                    'Status': 'Completed'
                })
            else:
                summary.append({
                    'Step': step_name,
                    'Duration': 'N/A',
                    'Status': 'In Progress'
                })
        return pd.DataFrame(summary)

# ============================================================================
# CHECKPOINT SYSTEM
# ============================================================================

class CheckpointManager:
    """Save and restore pipeline state"""
    
    def __init__(self, file_manager):
        self.file_manager = file_manager
        self.checkpoints = {}
    
    def save_checkpoint(self, name, data):
        """Save checkpoint data"""
        try:
            checkpoint_file = f"checkpoint_{name}_{datetime.now().strftime('%H%M%S')}.json"
            filepath = self.file_manager.save_file(data, checkpoint_file, 'checkpoints')
            self.checkpoints[name] = filepath
            logger.info(f"Saved checkpoint: {name}")
            return filepath
        except Exception as e:
            logger.error(f"Failed to save checkpoint {name}: {e}")
            return None
    
    def load_checkpoint(self, name):
        """Load checkpoint data"""
        try:
            if name in self.checkpoints:
                with open(self.checkpoints[name], 'r') as f:
                    data = json.load(f)
                logger.info(f"Loaded checkpoint: {name}")
                return data
        except Exception as e:
            logger.error(f"Failed to load checkpoint {name}: {e}")
        return None

# ============================================================================
# Initialize global objects
# ============================================================================

# These will be initialized when the analysis starts
file_manager = None
progress_tracker = None
checkpoint_manager = None

print("✅ Helper functions and exception handling framework initialized")
logger.info("Framework initialization complete")

## CELL 4: User Input Form and Configuration

In [ ]:
# ============================================================================
# USER INPUT FORM AND CONFIGURATION
# ============================================================================

@safe_cell_execution("User Input Form")
def create_input_form():
    """Create and display interactive input form"""
    
    logger.info("Creating user input form")
    
    # Create form widgets
    style = {'description_width': '150px'}
    layout = widgets.Layout(width='500px')
    
    # PDB ID input
    pdb_input = widgets.Text(
        value='1UBQ',
        placeholder='Enter PDB ID (e.g., 1UBQ)',
        description='PDB ID:',
        style=style,
        layout=layout
    )
    
    # E-value threshold
    evalue_input = widgets.FloatLogSlider(
        value=1e-5,
        min=-10,
        max=0,
        step=0.5,
        description='E-value threshold:',
        style=style,
        layout=layout
    )
    
    # Maximum sequences
    max_seqs_input = widgets.IntSlider(
        value=5000,
        min=100,
        max=10000,
        step=100,
        description='Max sequences:',
        style=style,
        layout=layout
    )
    
    # Coverage threshold
    coverage_input = widgets.IntSlider(
        value=50,
        min=10,
        max=100,
        step=5,
        description='Min coverage (%):',
        style=style,
        layout=layout
    )
    
    # Database selection
    database_input = widgets.Dropdown(
        options=['UniRef50 (Fast)', 'UniRef90 (Comprehensive)', 'UniRef100 (Complete)'],
        value='UniRef50 (Fast)',
        description='Database:',
        style=style,
        layout=layout
    )
    
    # Verbose mode
    verbose_input = widgets.Checkbox(
        value=True,
        description='Verbose logging',
        style=style
    )
    
    # Test mode
    test_mode_input = widgets.Checkbox(
        value=False,
        description='Test mode (small dataset)',
        style=style
    )
    
    # Output display
    output_display = widgets.Output()
    
    # Run button
    run_button = widgets.Button(
        description='Run Analysis',
        button_style='success',
        icon='play',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    # Validation function
    def validate_and_run(b):
        with output_display:
            output_display.clear_output()
            
            # Validate PDB ID
            pdb_id = pdb_input.value.strip().upper()
            if len(pdb_id) != 4:
                print("❌ Invalid PDB ID. Must be 4 characters (e.g., 1UBQ)")
                return
            
            # Store configuration
            global config
            config = {
                'pdb_id': pdb_id,
                'evalue': evalue_input.value,
                'max_sequences': max_seqs_input.value,
                'min_coverage': coverage_input.value,
                'database': database_input.value.split(' ')[0],
                'verbose': verbose_input.value,
                'test_mode': test_mode_input.value,
                'timestamp': datetime.now().strftime('%Y%m%d_%H%M%S')
            }
            
            # Initialize managers
            global file_manager, progress_tracker, checkpoint_manager
            file_manager = FileManager(config['pdb_id'], config['timestamp'])
            progress_tracker = ProgressTracker()
            checkpoint_manager = CheckpointManager(file_manager)
            
            # Save configuration
            config_file = file_manager.save_file(config, 'config.json')
            
            # Display configuration
            print("✅ Configuration validated and saved!")
            print("\n" + "="*50)
            print("ANALYSIS CONFIGURATION")
            print("="*50)
            for key, value in config.items():
                print(f"{key:15}: {value}")
            print("\nOutput directory: ", file_manager.base_dir)
            print("\n✅ Ready to proceed with analysis!")
            print("Run the next cells to start the pipeline.")
            
            logger.info(f"Configuration set: {config}")
    
    run_button.on_click(validate_and_run)
    
    # Create form layout
    form = widgets.VBox([
        widgets.HTML("<h3>🧬 Phylogenetic Analysis Configuration</h3>"),
        widgets.HTML("<hr>"),
        pdb_input,
        evalue_input,
        max_seqs_input,
        coverage_input,
        database_input,
        widgets.HBox([verbose_input, test_mode_input]),
        widgets.HTML("<hr>"),
        run_button,
        output_display
    ])
    
    return form

# Display the form
form = create_input_form()
display(form)

# Initialize config variable
config = None

## CELL 5: PDB Download and Sequence Extraction

In [ ]:
# ============================================================================
# PDB DOWNLOAD AND SEQUENCE EXTRACTION
# ============================================================================

@safe_cell_execution("PDB Download and Sequence Extraction")
def download_and_extract_sequences():
    """Download PDB file and extract sequences"""
    
    if not config:
        raise ValueError("Please run the configuration cell first!")
    
    progress_tracker.start_step('PDB Download')
    
    pdb_id = config['pdb_id']
    logger.info(f"Downloading PDB {pdb_id}")
    
    sequences = {}
    
    try:
        # Method 1: Use Biopython PDBList
        pdbl = PDBList(verbose=False)
        pdb_file = pdbl.retrieve_pdb_file(
            pdb_id, 
            pdir='/content/temp',
            file_format='pdb'
        )
        logger.info(f"Downloaded PDB file: {pdb_file}")
        
        # Parse PDB and extract sequences
        with open(pdb_file, 'r') as f:
            for record in SeqIO.parse(f, 'pdb-atom'):
                chain_id = record.id.split(':')[1] if ':' in record.id else record.id
                sequences[f"{pdb_id}_{chain_id}"] = str(record.seq)
                logger.debug(f"Extracted chain {chain_id}: {len(record.seq)} residues")
        
    except Exception as e:
        logger.warning(f"Method 1 failed: {e}, trying alternative method")
        
        # Method 2: Use REST API
        try:
            # Get structure info
            api_url = f"https://data.rcsb.org/rest/v1/core/entry/{pdb_id}"
            response = requests.get(api_url)
            response.raise_for_status()
            structure_info = response.json()
            
            # Get polymer sequences
            polymer_url = f"https://data.rcsb.org/rest/v1/core/polymer_entity/{pdb_id}/1"
            response = requests.get(polymer_url)
            response.raise_for_status()
            polymer_data = response.json()
            
            # Extract sequence
            if 'entity_poly' in polymer_data:
                sequence = polymer_data['entity_poly']['pdbx_seq_one_letter_code_can']
                sequences[f"{pdb_id}_A"] = sequence.replace('\n', '')
                logger.info(f"Retrieved sequence via API: {len(sequence)} residues")
        
        except Exception as e2:
            logger.error(f"Both methods failed: {e2}")
            raise Exception(f"Could not retrieve sequences for PDB {pdb_id}")
    
    progress_tracker.complete_step('PDB Download')
    progress_tracker.start_step('Sequence Extraction')
    
    # Save sequences
    if sequences:
        # Create FASTA file
        fasta_content = ""
        for seq_id, seq in sequences.items():
            fasta_content += f">{seq_id}\n{seq}\n"
        
        query_file = file_manager.save_file(fasta_content, 'query_sequences.fasta', 'sequences')
        
        # Save sequence metadata
        seq_metadata = []
        for seq_id, seq in sequences.items():
            seq_metadata.append({
                'Sequence_ID': seq_id,
                'Length': len(seq),
                'First_10_AA': seq[:10],
                'Last_10_AA': seq[-10:]
            })
        
        metadata_df = pd.DataFrame(seq_metadata)
        file_manager.save_file(metadata_df, 'query_metadata.csv', 'csv_outputs')
        
        # Display summary
        print(f"\n✅ Extracted {len(sequences)} sequence(s) from PDB {pdb_id}")
        display(metadata_df)
        
        # Save checkpoint
        checkpoint_manager.save_checkpoint('sequences', {
            'pdb_id': pdb_id,
            'sequences': sequences,
            'query_file': query_file
        })
        
        progress_tracker.complete_step('Sequence Extraction')
        
        logger.info(f"Successfully extracted {len(sequences)} sequences")
        return sequences, query_file
    
    else:
        raise Exception("No sequences extracted")

# Run extraction
sequences, query_file = download_and_extract_sequences()
print(f"\n✅ Query sequences saved to: {query_file}")

## CELL 6: MMseqs2 Homolog Search

In [ ]:
# ============================================================================
# MMSEQS2 HOMOLOG SEARCH
# ============================================================================

@safe_cell_execution("MMseqs2 Homolog Search")
def run_mmseqs2_search():
    """Run MMseqs2 search for homologs and paralogs"""
    
    if not config or not sequences:
        raise ValueError("Please run previous cells first!")
    
    progress_tracker.start_step('Homolog Search')
    
    logger.info(f"Starting MMseqs2 search with database {config['database']}")
    
    # Check if MMseqs2 is available
    mmseqs_bin = '/content/tools/mmseqs/bin/mmseqs'
    if not os.path.exists(mmseqs_bin):
        logger.warning("MMseqs2 binary not found, attempting to install")
        # Try to install MMseqs2
        try:
            os.makedirs('/content/tools', exist_ok=True)
            install_cmd = '''
            cd /content/tools && \
            wget -q https://github.com/soedinglab/MMseqs2/releases/latest/download/mmseqs-linux-avx2.tar.gz && \
            tar xzf mmseqs-linux-avx2.tar.gz
            '''
            subprocess.run(install_cmd, shell=True, check=True)
            logger.info("MMseqs2 installed successfully")
        except Exception as e:
            logger.error(f"Failed to install MMseqs2: {e}")
            print("⚠️ MMseqs2 not available, using mock data for demonstration")
            return generate_mock_homologs()
    
    # Download database if needed
    db_dir = '/content/databases'
    os.makedirs(db_dir, exist_ok=True)
    
    db_name = config['database']
    db_path = f'{db_dir}/{db_name}'
    
    if not os.path.exists(db_path):
        print(f"📥 Downloading {db_name} database (this may take a while)...")
        logger.info(f"Downloading {db_name} database")
        
        try:
            # For demo purposes, use a smaller database or subset
            if config['test_mode']:
                print("🧪 Test mode: Using small test database")
                # Create a small test database
                subprocess.run(
                    f'{mmseqs_bin} databases UniProtKB/Swiss-Prot {db_path} /tmp',
                    shell=True,
                    capture_output=True
                )
            else:
                # Download full database
                subprocess.run(
                    f'{mmseqs_bin} databases {db_name} {db_path} /tmp',
                    shell=True,
                    capture_output=True,
                    timeout=1800  # 30 minute timeout
                )
            
            logger.info(f"{db_name} database ready")
        except subprocess.TimeoutExpired:
            logger.warning("Database download timeout, using fallback")
            print("⚠️ Database download timeout, using smaller database")
            db_name = 'UniProtKB/Swiss-Prot'
            db_path = f'{db_dir}/SwissProt'
        except Exception as e:
            logger.error(f"Database download failed: {e}")
            print(f"⚠️ Could not download database: {e}")
            return generate_mock_homologs()
    
    # Run MMseqs2 search
    try:
        print("🔍 Running homolog search...")
        
        # Create query database
        query_db = '/content/temp/query_db'
        result_db = '/content/temp/result_db'
        
        # Convert FASTA to MMseqs2 database
        subprocess.run(
            f'{mmseqs_bin} createdb {query_file} {query_db}',
            shell=True,
            check=True,
            capture_output=True
        )
        
        # Run search
        search_params = [
            f'-e {config["evalue"]}',
            f'--max-seqs {config["max_sequences"]}',
            f'--min-seq-id 0.3',
            f'-c {config["min_coverage"]/100}',
            '--threads 4'
        ]
        
        search_cmd = f'{mmseqs_bin} search {query_db} {db_path} {result_db} /tmp {" ".join(search_params)}'
        
        logger.debug(f"Search command: {search_cmd}")
        
        result = subprocess.run(
            search_cmd,
            shell=True,
            capture_output=True,
            text=True,
            timeout=600  # 10 minute timeout
        )
        
        if result.returncode != 0:
            logger.warning(f"Search returned non-zero: {result.stderr}")
        
        # Convert results to FASTA
        result_fasta = '/content/temp/homologs.fasta'
        subprocess.run(
            f'{mmseqs_bin} result2flat {query_db} {db_path} {result_db} {result_fasta}',
            shell=True,
            check=True
        )
        
        # Parse results
        homologs = parse_mmseqs2_results(result_fasta)
        
        print(f"✅ Found {len(homologs)} homologous sequences")
        logger.info(f"Search complete: {len(homologs)} homologs found")
        
    except subprocess.TimeoutExpired:
        logger.warning("Search timeout, using partial results")
        print("⚠️ Search timeout, using partial results")
        homologs = generate_mock_homologs()
    except Exception as e:
        logger.error(f"Search failed: {e}")
        print(f"⚠️ Search failed: {e}")
        print("Using mock data for demonstration")
        homologs = generate_mock_homologs()
    
    # Save homolog data
    save_homolog_results(homologs)
    
    # Create visualizations
    create_homolog_visualizations(homologs)
    
    progress_tracker.complete_step('Homolog Search')
    
    return homologs

def parse_mmseqs2_results(result_file):
    """Parse MMseqs2 search results"""
    homologs = []
    
    try:
        if os.path.exists(result_file):
            with open(result_file, 'r') as f:
                for record in SeqIO.parse(f, 'fasta'):
                    homologs.append({
                        'id': record.id,
                        'sequence': str(record.seq),
                        'description': record.description
                    })
    except Exception as e:
        logger.error(f"Failed to parse results: {e}")
    
    return homologs

def generate_mock_homologs():
    """Generate mock homolog data for testing"""
    import random
    
    logger.info("Generating mock homolog data")
    
    # Get the first query sequence
    query_seq = list(sequences.values())[0]
    
    homologs = []
    organisms = ['Human', 'Mouse', 'Rat', 'Zebrafish', 'Drosophila', 'C.elegans', 'Yeast']
    
    # Add query sequence
    homologs.append({
        'id': f'{config["pdb_id"]}_query',
        'sequence': query_seq,
        'description': 'Query sequence',
        'organism': 'Query',
        'evalue': 0.0,
        'identity': 100.0
    })
    
    # Generate mock homologs with mutations
    for i in range(min(20, config['max_sequences'])):
        # Create sequence with random mutations
        seq_list = list(query_seq)
        num_mutations = random.randint(5, len(seq_list)//3)
        
        for _ in range(num_mutations):
            pos = random.randint(0, len(seq_list)-1)
            aa_choices = 'ACDEFGHIKLMNPQRSTVWY'
            seq_list[pos] = random.choice(aa_choices)
        
        homologs.append({
            'id': f'homolog_{i+1}',
            'sequence': ''.join(seq_list),
            'description': f'{random.choice(organisms)} protein homolog',
            'organism': random.choice(organisms),
            'evalue': 10**(random.uniform(-10, -3)),
            'identity': random.uniform(30, 95)
        })
    
    return homologs

def save_homolog_results(homologs):
    """Save homolog search results"""
    
    # Save as FASTA
    fasta_content = ""
    for h in homologs:
        fasta_content += f">{h['id']} {h.get('description', '')}\n{h['sequence']}\n"
    
    homolog_fasta = file_manager.save_file(fasta_content, 'homologs.fasta', 'sequences')
    
    # Save summary CSV
    summary_data = []
    for h in homologs:
        summary_data.append({
            'ID': h['id'],
            'Description': h.get('description', ''),
            'Length': len(h['sequence']),
            'E-value': h.get('evalue', 'N/A'),
            'Identity_%': h.get('identity', 'N/A'),
            'Organism': h.get('organism', 'Unknown')
        })
    
    summary_df = pd.DataFrame(summary_data)
    file_manager.save_file(summary_df, 'homolog_summary.csv', 'csv_outputs')
    
    # Display top results
    print("\n📊 Top 10 Homologs:")
    display(summary_df.head(10))
    
    # Save checkpoint
    checkpoint_manager.save_checkpoint('homologs', {
        'count': len(homologs),
        'homolog_fasta': homolog_fasta
    })

def create_homolog_visualizations(homologs):
    """Create visualizations for homolog search results"""
    
    if len(homologs) == 0:
        return
    
    # E-value distribution
    if any('evalue' in h for h in homologs):
        evalues = [h.get('evalue', 1e-3) for h in homologs if h.get('evalue')]
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        
        # E-value distribution (log scale)
        ax1.hist(np.log10(evalues), bins=30, edgecolor='black')
        ax1.set_xlabel('log10(E-value)')
        ax1.set_ylabel('Count')
        ax1.set_title('E-value Distribution')
        ax1.grid(True, alpha=0.3)
        
        # Identity distribution
        if any('identity' in h for h in homologs):
            identities = [h.get('identity', 50) for h in homologs if h.get('identity')]
            ax2.hist(identities, bins=30, edgecolor='black', color='green')
            ax2.set_xlabel('Identity (%)')
            ax2.set_ylabel('Count')
            ax2.set_title('Sequence Identity Distribution')
            ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(file_manager.dirs['visualizations'] + '/homolog_distributions.png', dpi=150)
        plt.show()
    
    # Organism distribution pie chart
    if any('organism' in h for h in homologs):
        organisms = [h.get('organism', 'Unknown') for h in homologs]
        org_counts = pd.Series(organisms).value_counts()
        
        if len(org_counts) > 0:
            fig, ax = plt.subplots(figsize=(10, 8))
            colors = plt.cm.Set3(np.linspace(0, 1, len(org_counts)))
            ax.pie(org_counts.values, labels=org_counts.index, autopct='%1.1f%%', colors=colors)
            ax.set_title('Organism Distribution in Homologs')
            plt.savefig(file_manager.dirs['visualizations'] + '/organism_distribution.png', dpi=150)
            plt.show()

# Run homolog search
homologs = run_mmseqs2_search()
print(f"\n✅ Homolog search complete! Found {len(homologs)} sequences.")

## CELL 7: Multiple Sequence Alignment with FAMSA

In [ ]:
# ============================================================================
# MULTIPLE SEQUENCE ALIGNMENT WITH FAMSA
# ============================================================================

@safe_cell_execution("Multiple Sequence Alignment")
def perform_alignment():
    """Perform multiple sequence alignment using FAMSA or fallback methods"""
    
    if not homologs:
        raise ValueError("Please run homolog search first!")
    
    progress_tracker.start_step('Alignment')
    
    logger.info(f"Starting alignment with {len(homologs)} sequences")
    print(f"🧬 Aligning {len(homologs)} sequences...")
    
    alignment = None
    
    # Method 1: Try pyfamsa
    try:
        import pyfamsa
        logger.info("Using pyfamsa for alignment")
        
        # Convert homologs to pyfamsa format
        sequences_list = []
        for h in homologs:
            sequences_list.append(pyfamsa.Sequence(
                h['id'].encode(),
                h['sequence'].encode()
            ))
        
        # Run FAMSA alignment
        aligner = pyfamsa.Aligner(threads=4)
        msa = aligner.align(sequences_list)
        
        # Convert to Biopython alignment
        from Bio.Align import MultipleSeqAlignment
        from Bio.SeqRecord import SeqRecord
        from Bio.Seq import Seq
        
        aligned_records = []
        for seq in msa:
            aligned_records.append(SeqRecord(
                Seq(seq.sequence.decode()),
                id=seq.id.decode(),
                description=''
            ))
        
        alignment = MultipleSeqAlignment(aligned_records)
        logger.info("FAMSA alignment successful")
        
    except ImportError:
        logger.warning("pyfamsa not available, trying alternative methods")
        
        # Method 2: Try MAFFT via subprocess
        try:
            logger.info("Attempting MAFFT alignment")
            
            # Save sequences to temp file
            temp_input = '/content/temp/input.fasta'
            temp_output = '/content/temp/aligned.fasta'
            
            with open(temp_input, 'w') as f:
                for h in homologs:
                    f.write(f">{h['id']}\n{h['sequence']}\n")
            
            # Try to install MAFFT if not present
            mafft_check = subprocess.run('which mafft', shell=True, capture_output=True)
            if mafft_check.returncode != 0:
                subprocess.run('apt-get update && apt-get install -y mafft', shell=True)
            
            # Run MAFFT
            subprocess.run(
                f'mafft --auto --thread 4 {temp_input} > {temp_output}',
                shell=True,
                check=True
            )
            
            # Load alignment
            alignment = AlignIO.read(temp_output, 'fasta')
            logger.info("MAFFT alignment successful")
            
        except Exception as e:
            logger.warning(f"MAFFT failed: {e}, using simple alignment")
            
            # Method 3: Simple pairwise alignment fallback
            alignment = create_simple_alignment(homologs)
    
    except Exception as e:
        logger.error(f"Alignment failed: {e}")
        print(f"⚠️ Alignment error: {e}")
        # Use simple alignment as last resort
        alignment = create_simple_alignment(homologs)
    
    if alignment:
        # Save alignment
        alignment_file = file_manager.dirs['alignments'] + '/alignment.fasta'
        AlignIO.write(alignment, alignment_file, 'fasta')
        logger.info(f"Alignment saved to {alignment_file}")
        
        # Calculate alignment statistics
        stats = calculate_alignment_stats(alignment)
        
        # Save statistics
        stats_df = pd.DataFrame([stats])
        file_manager.save_file(stats_df, 'alignment_statistics.csv', 'csv_outputs')
        
        # Display statistics
        print("\n📊 Alignment Statistics:")
        print(f"  • Number of sequences: {alignment.get_alignment_length()}")
        print(f"  • Alignment length: {len(alignment[0])}")
        print(f"  • Conservation score: {stats['conservation']:.2%}")
        print(f"  • Gap percentage: {stats['gap_percentage']:.2%}")
        
        # Create alignment visualization
        visualize_alignment(alignment)
        
        # Save checkpoint
        checkpoint_manager.save_checkpoint('alignment', {
            'num_sequences': len(alignment),
            'alignment_length': alignment.get_alignment_length(),
            'alignment_file': alignment_file
        })
        
        progress_tracker.complete_step('Alignment')
        
        return alignment
    
    else:
        raise Exception("Failed to create alignment")

def create_simple_alignment(homologs):
    """Create a simple alignment for fallback"""
    from Bio.Align import MultipleSeqAlignment
    from Bio.SeqRecord import SeqRecord
    from Bio.Seq import Seq
    
    logger.info("Creating simple alignment as fallback")
    
    # Find the maximum sequence length
    max_len = max(len(h['sequence']) for h in homologs)
    
    # Pad sequences to same length
    aligned_records = []
    for h in homologs:
        padded_seq = h['sequence'] + '-' * (max_len - len(h['sequence']))
        aligned_records.append(SeqRecord(
            Seq(padded_seq),
            id=h['id'],
            description=''
        ))
    
    return MultipleSeqAlignment(aligned_records)

def calculate_alignment_stats(alignment):
    """Calculate alignment statistics"""
    
    stats = {}
    
    # Get alignment dimensions
    num_seqs = len(alignment)
    aln_length = alignment.get_alignment_length()
    
    # Calculate conservation
    conservation_scores = []
    gap_counts = []
    
    for i in range(aln_length):
        column = alignment[:, i]
        
        # Count gaps
        gap_count = column.count('-')
        gap_counts.append(gap_count)
        
        # Calculate conservation (excluding gaps)
        non_gap_chars = [c for c in column if c != '-']
        if non_gap_chars:
            most_common = max(set(non_gap_chars), key=non_gap_chars.count)
            conservation = non_gap_chars.count(most_common) / len(non_gap_chars)
            conservation_scores.append(conservation)
    
    stats['num_sequences'] = num_seqs
    stats['alignment_length'] = aln_length
    stats['conservation'] = np.mean(conservation_scores) if conservation_scores else 0
    stats['gap_percentage'] = np.mean(gap_counts) / num_seqs if gap_counts else 0
    stats['highly_conserved_positions'] = sum(1 for s in conservation_scores if s > 0.9)
    
    return stats

def visualize_alignment(alignment):
    """Create alignment visualization"""
    
    # Conservation plot
    aln_length = alignment.get_alignment_length()
    conservation_scores = []
    
    for i in range(min(aln_length, 500)):  # Limit to first 500 positions for visualization
        column = alignment[:, i]
        non_gap_chars = [c for c in column if c != '-']
        if non_gap_chars:
            most_common = max(set(non_gap_chars), key=non_gap_chars.count)
            conservation = non_gap_chars.count(most_common) / len(non_gap_chars)
            conservation_scores.append(conservation)
        else:
            conservation_scores.append(0)
    
    # Create conservation plot
    fig, ax = plt.subplots(figsize=(14, 4))
    positions = range(len(conservation_scores))
    ax.plot(positions, conservation_scores, linewidth=1)
    ax.fill_between(positions, conservation_scores, alpha=0.3)
    ax.set_xlabel('Position')
    ax.set_ylabel('Conservation Score')
    ax.set_title('Alignment Conservation Profile')
    ax.grid(True, alpha=0.3)
    ax.set_ylim([0, 1])
    
    # Add threshold line
    ax.axhline(y=0.9, color='r', linestyle='--', alpha=0.5, label='High conservation (>90%)')
    ax.legend()
    
    plt.tight_layout()
    plt.savefig(file_manager.dirs['visualizations'] + '/alignment_conservation.png', dpi=150)
    plt.show()
    
    # Create a heatmap of the first 20 sequences and 100 positions
    if len(alignment) > 1:
        # Convert alignment to numeric matrix for heatmap
        aa_to_num = {aa: i for i, aa in enumerate('ACDEFGHIKLMNPQRSTVWY-')}
        
        num_seqs_show = min(20, len(alignment))
        num_pos_show = min(100, alignment.get_alignment_length())
        
        matrix = np.zeros((num_seqs_show, num_pos_show))
        
        for i in range(num_seqs_show):
            for j in range(num_pos_show):
                aa = alignment[i].seq[j]
                matrix[i, j] = aa_to_num.get(aa, 20)
        
        # Create heatmap
        fig, ax = plt.subplots(figsize=(14, 6))
        im = ax.imshow(matrix, cmap='viridis', aspect='auto')
        ax.set_xlabel('Position')
        ax.set_ylabel('Sequence')
        ax.set_title('Alignment Heatmap (subset)')
        
        # Add sequence IDs as y-axis labels
        seq_ids = [alignment[i].id[:20] for i in range(num_seqs_show)]
        ax.set_yticks(range(num_seqs_show))
        ax.set_yticklabels(seq_ids, fontsize=8)
        
        plt.colorbar(im, ax=ax, label='Amino Acid')
        plt.tight_layout()
        plt.savefig(file_manager.dirs['visualizations'] + '/alignment_heatmap.png', dpi=150)
        plt.show()

# Perform alignment
alignment = perform_alignment()
print("\n✅ Alignment complete!")

## CELL 8: Phylogenetic Tree Construction with IQ-TREE

In [ ]:
# ============================================================================
# PHYLOGENETIC TREE CONSTRUCTION WITH IQ-TREE
# ============================================================================

@safe_cell_execution("Phylogenetic Tree Construction")
def construct_phylogenetic_tree():
    """Construct phylogenetic tree using IQ-TREE or alternatives"""
    
    if not alignment:
        raise ValueError("Please run alignment first!")
    
    progress_tracker.start_step('Tree Construction')
    
    logger.info("Starting phylogenetic tree construction")
    print("🌳 Constructing phylogenetic tree...")
    
    tree = None
    
    # Save alignment for tree construction
    aln_file = '/content/temp/alignment_for_tree.fasta'
    AlignIO.write(alignment, aln_file, 'fasta')
    
    # Method 1: Try IQ-TREE
    iqtree_bin = '/content/tools/iqtree2'
    if os.path.exists(iqtree_bin):
        try:
            logger.info("Using IQ-TREE for tree construction")
            
            # Run IQ-TREE with ModelFinder and UFBoot
            iqtree_cmd = [
                iqtree_bin,
                '-s', aln_file,
                '-m', 'MFP',  # ModelFinder
                '-bb', '1000',  # UFBoot with 1000 replicates
                '-alrt', '1000',  # SH-aLRT test
                '-nt', '4',  # Number of threads
                '-pre', '/content/temp/iqtree'
            ]
            
            result = subprocess.run(
                iqtree_cmd,
                capture_output=True,
                text=True,
                timeout=600
            )
            
            if result.returncode == 0:
                # Load the tree
                tree_file = '/content/temp/iqtree.treefile'
                if os.path.exists(tree_file):
                    tree = Phylo.read(tree_file, 'newick')
                    logger.info("IQ-TREE completed successfully")
                    
                    # Parse IQ-TREE report for best model
                    report_file = '/content/temp/iqtree.iqtree'
                    if os.path.exists(report_file):
                        with open(report_file, 'r') as f:
                            report_content = f.read()
                            if 'Best-fit model:' in report_content:
                                best_model = report_content.split('Best-fit model:')[1].split('\n')[0].strip()
                                print(f"📊 Best-fit model: {best_model}")
            else:
                logger.warning(f"IQ-TREE failed: {result.stderr}")
                
        except subprocess.TimeoutExpired:
            logger.warning("IQ-TREE timeout")
        except Exception as e:
            logger.error(f"IQ-TREE error: {e}")
    
    # Method 2: Try FastTree
    if tree is None:
        try:
            logger.info("Trying FastTree")
            
            # Install FastTree if needed
            fasttree_check = subprocess.run('which fasttree', shell=True, capture_output=True)
            if fasttree_check.returncode != 0:
                subprocess.run('apt-get update && apt-get install -y fasttree', shell=True)
            
            # Run FastTree
            tree_file = '/content/temp/fasttree.tree'
            subprocess.run(
                f'fasttree {aln_file} > {tree_file}',
                shell=True,
                check=True
            )
            
            if os.path.exists(tree_file):
                tree = Phylo.read(tree_file, 'newick')
                logger.info("FastTree completed successfully")
                
        except Exception as e:
            logger.error(f"FastTree failed: {e}")
    
    # Method 3: Use Biopython's distance-based method
    if tree is None:
        try:
            logger.info("Using Biopython distance-based tree construction")
            
            from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
            
            # Calculate distance matrix
            calculator = DistanceCalculator('identity')
            dm = calculator.get_distance(alignment)
            
            # Construct tree using UPGMA
            constructor = DistanceTreeConstructor()
            tree = constructor.upgma(dm)
            
            logger.info("Distance-based tree construction successful")
            
        except Exception as e:
            logger.error(f"Distance-based construction failed: {e}")
            print(f"❌ Could not construct tree: {e}")
            return None
    
    if tree:
        # Save tree
        tree_newick = file_manager.dirs['trees'] + '/phylogenetic_tree.newick'
        Phylo.write(tree, tree_newick, 'newick')
        logger.info(f"Tree saved to {tree_newick}")
        
        # Calculate tree statistics
        stats = calculate_tree_stats(tree)
        
        # Save tree statistics
        stats_df = pd.DataFrame([stats])
        file_manager.save_file(stats_df, 'tree_statistics.csv', 'csv_outputs')
        
        # Display tree statistics
        print("\n📊 Tree Statistics:")
        print(f"  • Number of terminals: {stats['num_terminals']}")
        print(f"  • Number of internal nodes: {stats['num_internal']}")
        print(f"  • Total branch length: {stats['total_branch_length']:.4f}")
        print(f"  • Tree depth: {stats['max_depth']:.4f}")
        
        # Visualize tree
        visualize_tree(tree)
        
        # Save checkpoint
        checkpoint_manager.save_checkpoint('tree', {
            'tree_file': tree_newick,
            'statistics': stats
        })
        
        progress_tracker.complete_step('Tree Construction')
        
        return tree
    
    else:
        raise Exception("Failed to construct phylogenetic tree")

def calculate_tree_stats(tree):
    """Calculate tree statistics"""
    
    stats = {}
    
    # Count terminals and internal nodes
    terminals = tree.get_terminals()
    all_clades = list(tree.find_clades())
    
    stats['num_terminals'] = len(terminals)
    stats['num_internal'] = len(all_clades) - len(terminals)
    
    # Calculate total branch length
    total_length = sum(clade.branch_length for clade in all_clades if clade.branch_length)
    stats['total_branch_length'] = total_length
    
    # Calculate tree depth
    depths = [tree.distance(terminal) for terminal in terminals]
    stats['max_depth'] = max(depths) if depths else 0
    stats['avg_depth'] = np.mean(depths) if depths else 0
    
    return stats

def visualize_tree(tree):
    """Create tree visualizations"""
    
    # Basic tree plot using matplotlib
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 10))
    
    # Rectangular tree
    Phylo.draw(tree, axes=ax1, do_show=False)
    ax1.set_title('Phylogenetic Tree (Rectangular)')
    ax1.set_xlabel('Branch Length')
    
    # Circular tree
    try:
        Phylo.draw_graphviz(tree, axes=ax2, prog='neato')
        ax2.set_title('Phylogenetic Tree (Circular)')
    except:
        # If graphviz not available, draw another rectangular
        Phylo.draw(tree, axes=ax2, do_show=False)
        ax2.set_title('Phylogenetic Tree (Alternative View)')
    
    plt.tight_layout()
    plt.savefig(file_manager.dirs['visualizations'] + '/phylogenetic_trees.png', dpi=200)
    plt.show()
    
    # Create an interactive tree plot using plotly
    try:
        create_interactive_tree(tree)
    except Exception as e:
        logger.warning(f"Could not create interactive tree: {e}")

def create_interactive_tree(tree):
    """Create an interactive tree visualization with plotly"""
    
    # Get coordinates for tree nodes
    def get_coords(clade, x_parent=0, depth=0):
        coords = []
        y = depth
        
        if clade.branch_length:
            x = x_parent + clade.branch_length
        else:
            x = x_parent + 0.1
        
        coords.append((x, y, clade.name if clade.name else ''))
        
        for i, child in enumerate(clade.clades):
            child_coords = get_coords(child, x, depth - (i+1)*0.5)
            coords.extend(child_coords)
        
        return coords
    
    coords = get_coords(tree.root)
    
    # Create plotly figure
    fig = go.Figure()
    
    # Add tree branches
    for coord in coords:
        x, y, label = coord
        fig.add_trace(go.Scatter(
            x=[x], y=[y],
            mode='markers+text',
            text=[label],
            textposition='middle right',
            marker=dict(size=5),
            showlegend=False
        ))
    
    fig.update_layout(
        title='Interactive Phylogenetic Tree',
        xaxis_title='Evolutionary Distance',
        yaxis_title='',
        height=800,
        hovermode='closest'
    )
    
    # Save interactive plot
    fig.write_html(file_manager.dirs['visualizations'] + '/interactive_tree.html')
    fig.show()

# Construct tree
tree = construct_phylogenetic_tree()
print("\n✅ Phylogenetic tree construction complete!")

## CELL 9: Final Report and Summary

In [ ]:
# ============================================================================
# FINAL REPORT AND SUMMARY
# ============================================================================

@safe_cell_execution("Final Report Generation")
def generate_final_report():
    """Generate comprehensive final report"""
    
    progress_tracker.start_step('Report Generation')
    
    logger.info("Generating final report")
    print("📝 Generating final report...")
    
    # Compile all results
    report = {
        'Analysis Summary': {
            'PDB ID': config['pdb_id'],
            'Analysis Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'Database Used': config['database'],
            'E-value Threshold': config['evalue'],
            'Output Directory': file_manager.base_dir
        },
        'Results': {
            'Query Sequences': len(sequences),
            'Homologs Found': len(homologs) if homologs else 0,
            'Alignment Length': alignment.get_alignment_length() if alignment else 0,
            'Tree Terminals': len(tree.get_terminals()) if tree else 0
        }
    }
    
    # Save comprehensive report
    report_file = file_manager.save_file(report, 'analysis_report.json')
    
    # Create summary CSV
    summary_data = []
    for category, items in report.items():
        for key, value in items.items():
            summary_data.append({
                'Category': category,
                'Metric': key,
                'Value': str(value)
            })
    
    summary_df = pd.DataFrame(summary_data)
    file_manager.save_file(summary_df, 'analysis_summary.csv', 'csv_outputs')
    
    # Get execution summary
    execution_summary = progress_tracker.get_summary()
    file_manager.save_file(execution_summary, 'execution_summary.csv', 'csv_outputs')
    
    # Display final summary
    print("\n" + "="*70)
    print("PHYLOGENETIC ANALYSIS COMPLETE")
    print("="*70)
    
    print("\n📊 Analysis Summary:")
    for category, items in report.items():
        print(f"\n{category}:")
        for key, value in items.items():
            print(f"  • {key}: {value}")
    
    print("\n⏱️ Execution Times:")
    display(execution_summary)
    
    print("\n📁 Output Files:")
    for dir_name, dir_path in file_manager.dirs.items():
        files = os.listdir(dir_path)
        if files:
            print(f"\n  {dir_name}:")
            for f in files[:5]:  # Show first 5 files
                print(f"    - {f}")
            if len(files) > 5:
                print(f"    ... and {len(files)-5} more files")
    
    # Create download links if in Colab
    if IN_COLAB:
        print("\n💾 Download Results:")
        print(f"Results saved to: {file_manager.base_dir}")
        
        # Create a zip file of all results
        try:
            import zipfile
            zip_path = f'/content/{config["pdb_id"]}_phylogenetic_results.zip'
            
            with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for root, dirs, files in os.walk(file_manager.base_dir):
                    for file in files:
                        file_path = os.path.join(root, file)
                        arcname = os.path.relpath(file_path, file_manager.base_dir)
                        zipf.write(file_path, arcname)
            
            print(f"\n📦 Results archived: {zip_path}")
            
            # Offer download
            from google.colab import files
            print("\n⬇️ Download the complete results archive:")
            files.download(zip_path)
            
        except Exception as e:
            logger.error(f"Could not create zip archive: {e}")
    
    progress_tracker.complete_step('Report Generation')
    
    # Final log message
    logger.info("Analysis pipeline completed successfully")
    
    print("\n" + "="*70)
    print("✅ ANALYSIS COMPLETE!")
    print("="*70)
    print("\nThank you for using the Phylogenetic Analysis Pipeline!")
    print("Results have been saved to your Google Drive.")
    
    return report

# Generate final report
final_report = generate_final_report()